In [33]:
import votekit.ballot_generator.std_generator.spacial as spacial
import votekit.ballot_generator as bg
import numpy as np

class VoterType:
    def __init__(self, generator,name=""):
        self.gen = generator
        self.name = name

    def uniform_centered_L1(x,y,r=1,name=""):
        gen = lambda: np.random.uniform(low= (x-r,y-r),high= (x+r,y+r),size= 2)
        return VoterType(gen,name)

class Region:
    def __init__(self,voter_types,amounts,name=""):
        self.voter_types=voter_types
        self.amounts = amounts
        assert len(self.voter_types) == len(self.amounts), "Dude What"
        
        tot = sum(amounts)
        self.proportions = [x/tot for x in amounts]

    def gen_one_random(self):
        my_guy = np.random.choice(self.voter_types, p=self.proportions)
        return my_guy.gen()

def voter_generator_R_sep(R=0):
    first_rand = {"low": (-1,-1),"high": (1,1), "size": 2}
    second_rand = {"low": (-1,R-1),"high": (1,R+1), "size": 2}
    choose = np.random.rand(1)
    if choose < 1/2:
        return np.random.uniform(**first_rand)
    else:
        return np.random.uniform(**second_rand)

def candidate_generator_R_sep(R=0):
    return np.random.uniform(**{"low": (-1,-1), "high": (-1,R+1), "size": 2})
    

In [36]:
voter_type_K = VoterType.uniform_centered_L1(0,0,name="K")
voter_type_L = VoterType.uniform_centered_L1(2,2,name="K")
region_1 = Region([voter_type_K,voter_type_L],[5,20])

bg.spacial_profile_and_positions_generator(
    number_of_ballots= 30,
    candidates= ["A","B"],
    voter_dist=region_1.gen_one_random,
    candidate_dist = np.random.uniform,
    candidate_dist_kwargs={"low": (-1,-1), "high": (-1,+1), "size": 2})

(RankProfile
 Maximum ranking length: 2
 Candidates: ('A', 'B')
 Candidates who received votes: ('A', 'B')
 Total number of Ballot objects: 30
 Total weight of Ballot objects: 30,
 {'A': array([-1.        , -0.14155572]),
  'B': array([-1.        , -0.50687387])},
 array([[ 0.84975888,  0.41194465],
        [ 1.44094924,  1.28807454],
        [ 1.42123045,  1.58048431],
        [ 2.84475836,  1.54830424],
        [ 1.68187575,  2.82427061],
        [ 2.86472145,  1.06340657],
        [ 1.53419035,  1.26391096],
        [ 2.66611026,  1.59386641],
        [-0.19737835, -0.11704301],
        [ 0.77787112,  0.02320332],
        [ 2.57035911,  2.65023878],
        [ 1.71582101,  1.93607198],
        [ 0.80326957,  0.39765781],
        [ 1.00770546,  2.39425516],
        [ 1.93087965,  1.57175534],
        [-0.28389041, -0.58998399],
        [ 1.64375497,  1.06991397],
        [ 1.68188276,  1.71749599],
        [ 2.59477745,  2.52082055],
        [ 1.56846648,  1.2070784 ],
        [ 2.997